# xarray-enabled implementations of parcel functions

[MetPy](https://unidata.github.io/MetPy/latest/index.html) contains functions to calculate properties of lifted air parcels, but these functions work only on 1-dimensional arrays. Using a loop to apply these functions over every pixel in a large data set is slow. This package contains xarray-enabled implementations of the metpy parcel functions. This notebook shows results of testing the xarray implementations.

In [1]:
# Change to git working directory.
%cd git/xarray_parcel

/home/561/tr2908/git/xarray_parcel


In [2]:
import xarray
import warnings
import metpy
import metpy.plots
from metpy.units import units
import numpy as np
import matplotlib.pyplot as plt
import modules.parcel_functions as parcel
import modules.parcel_test as parcel_test
import modules.unit_tests as tests
from importlib import reload
from dask.distributed import Client, LocalCluster

In [3]:
## Local cluster (uses all resources of node running this notebook).
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)  

2022-07-07 11:02:53,649 - distributed.diskutils - INFO - Found stale lock file and directory '/local/up6/tr2908/tmp/dask-worker-space/worker-naxd2f33', purging
2022-07-07 11:02:53,649 - distributed.diskutils - INFO - Found stale lock file and directory '/local/up6/tr2908/tmp/dask-worker-space/worker-f4ufa1t9', purging
2022-07-07 11:02:53,650 - distributed.diskutils - INFO - Found stale lock file and directory '/local/up6/tr2908/tmp/dask-worker-space/worker-n6cblmr8', purging
2022-07-07 11:02:53,651 - distributed.diskutils - INFO - Found stale lock file and directory '/local/up6/tr2908/tmp/dask-worker-space/worker-gx1bjq1u', purging


# Settings

In [4]:
plt.rcParams['figure.figsize'] = (16,5)                   # Figure size for non-facetted plots.
plt.rcParams['font.size'] = 14                            # Font size for plots.
plt.rcParams['axes.formatter.useoffset'] = False          # Don't use offsets in plots.      
warnings.filterwarnings("ignore", category=FutureWarning) # Ignore FutureWarnings (in Dask). 

# Test data

The test data is a subset from the [Aus400](http://climate-cms.wikis.unsw.edu.au/Aus400) simulations (released under [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/)), containing pressure/temperature/specific humidity at 90 vertical layers for ~10k points.

In [132]:
dat = xarray.open_dataset('test_data.nc', chunks=200)

# A lookup table for moist lapse rates

The calculation of moist lapse rates is usually iterative. MetPy's `moist_lapse` function uses the SciPy function `odeint`, which is difficult to run in a vectorised format. I have provided a function `moist_lapse_analytical` which attempts to do the iterative calcultion, but it remains slow. To get around this bottleneck, the xarray implementation of `moist_lapse` uses lookup tables of moist adiabats which can be quickly returned and offer an acceptable level of accuracy. The lookup tables are generated by the function `moist_adiabat_tables`. Two lookup tables are returned; one that contains all the adiabats generated, the other that contains an index of which adiabat passes through given combinations of pressure/temperature. The lookup tables are cached on disk for fast access.

Before running any parcel functions that use moist lapse rates, the function `load_moist_adiabat_lookups` must be run to load the cached versions of the lookup tables.

In [6]:
parcel.load_moist_adiabat_lookups()

The index lookup table (`moist_adiabat_lookup` here) contains a grid of pressure/temperature combinations and identifies the index of the adiabat that passes through those combinations.

In [ ]:
parcel.moist_adiabat_lookup.adiabat.plot(y='pressure', yincrease=False, 
                                         cbar_kwargs={'label': 'Adiabat index'})
plt.show()

Interpolation of the adiabats across the discrete pressure/temperature fields means that pressure/temperature combinations which do not have any associated adiabat (yellow in the plot below) are limited to areas outside reasonable temperature ranges.

In [ ]:
np.isnan(parcel.moist_adiabat_lookup.adiabat).plot(y='pressure', yincrease=False,
                                                   cbar_kwargs={'label': 'Adiabat missing'})
plt.show()

The following plot shows output from the xarray lookup table implementation and the metpy implementation for a starting parcel at 1000 hPa and a variety of starting parcel temperatures. The adiabats are indistinguishable on the Skew T plot yet there are small differences introduced by the lookup table approach; the maximum difference for these adiabats is reported below.

In [ ]:
skew = metpy.plots.SkewT()
max_diff = 0
pressures = np.arange(1000, 100, step=-1)
for temperature in np.arange(250, 314, step=1):
    adiabat_xr = parcel.moist_lapse(pressure=pressures, parcel_pressure=1000, parcel_temperature=temperature)
    adiabat_mp = metpy.calc.moist_lapse(pressure=pressures*units.hPa, temperature=temperature*units.K, 
                                        reference_pressure=1000*units.hPa)
    max_diff = np.max([max_diff, np.round(np.max(np.abs(adiabat_xr.values - adiabat_mp.m)), 3)])
    skew.plot(pressures*units.hPa, adiabat_xr.values*units.K, 'r')
    skew.plot(pressures*units.hPa, adiabat_mp, 'b')
    
print(f'Maximum difference between metpy and xarray on these adiabats is {max_diff} K.')

# Testing

The following code takes a set of 225 data points and runs the xarray implementation (with virtual temperature correction turned off) and metpy implementations for a variety of functions. Any differences greater than a threshold of absolute difference of `1e-5` are reported. Note the maximum absolute difference and the maximum relative difference may be on different points.

In [133]:
# Take a sample of points from dat.
num_points = 15
lat = np.random.randint(0, dat.latitude.size-num_points)
lon = np.random.randint(0, dat.longitude.size-num_points)
pts = dat.isel(latitude=slice(lat, lat+num_points), 
               longitude=slice(lon, lon+num_points)).load()

## New code for wet bulb temperature calculation

In [138]:
dat = pts.load()
dat['dewpoint'] = metpy.calc.dewpoint_from_specific_humidity(
    pressure=dat.pressure,
    temperature=dat.temperature,
    specific_humidity=dat.specific_humidity)
dat['dewpoint'] = dat.dewpoint.metpy.convert_units('K')
dat['dewpoint'] = dat.dewpoint.metpy.dequantify()

In [141]:
dat = dat.chunk()

In [135]:
reload(parcel)
reload(parcel_test)

<module 'modules.parcel_test' from '/home/561/tr2908/git/xarray_parcel/modules/parcel_test.py'>

In [139]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [142]:
%memit
dat['wb'] = parcel.wet_bulb_temperature(pressure=dat.pressure, temperature=dat.temperature, dewpoint=dat.dewpoint)

peak memory: 1133.56 MiB, increment: 0.04 MiB


In [146]:
cluster.scale(0)
cluster.close()

# End testing code

In [97]:
xr, mp = parcel_test.test_parcel_functions(dat=pts.load(), virt_temp=False, lcl_interp='linear')

Calculating xarray results...		3.7218869930075016 s.
Calculating metpy serial results...	14.093373129027896 s.
Differences                                                       Max abs. diff        Max rel. diff       
Mixed parcel temperature [K] (mp_temperature)                     8e-05 K              0.0%                
Mixed-parcel dewpoint [kelvin] (mp_dewpoint)                      2e-05 kelvin         0.0%                
Moist lapse rate temperature [K] (moist_lapse_temp)               0.06534 K            0.09%               
Temperature at LCL [K] (surface_profile)                          0.06337 K            0.09%               
Level of free convection pressure [hPa] (surface_lfc_pressure)    1.09004 hPa          0.12%               
Lifting condensation level pressure [hPa] (surface_lcl_pressure)  0.00029 hPa          0.0%                
Lifting condensation level temperature [K] (surface_lcl_temp)     2e-05 K              0.0%                
Level of free convection 

As outlined above, the new implementation of `moist_lapse` is slightly less accurate than MetPy's analytical function. These changes introduce some differences to the other calculations. To see testing results without this difference's impact, here are the tests again but this time using MetPy's approach to calculate moist lapse rates.

In [122]:
reload(parcel)
reload(parcel_test)

parcel.moist_lapse = parcel_test.moist_lapse_serial
_, _ = parcel_test.test_parcel_functions(dat=pts, virt_temp=False)

Calculating xarray results...		4.516456968995044 s.
Calculating metpy serial results...	0.8662842340127099 s.
Differences                                                       Max abs. diff        Max rel. diff       
Mixed parcel temperature [K] (mp_temperature)                     2e-05 K              0.0%                
Mixed-parcel dewpoint [kelvin] (mp_dewpoint)                      1e-05 kelvin         0.0%                
Temperature at LCL [K] (surface_profile)                          3e-05 K              0.0%                
Level of free convection pressure [hPa] (surface_lfc_pressure)    0.00075 hPa          0.0%                
Lifting condensation level pressure [hPa] (surface_lcl_pressure)  0.00029 hPa          0.0%                
Lifting condensation level temperature [K] (surface_lcl_temp)     2e-05 K              0.0%                
Level of free convection temperature [K] (surface_lfc_temp)       2e-05 K              0.0%                
Equilibrium level pressure

Another function in which small differences occur is `lcl`, which determines lifting condensation levels. In both cases the MetPy function is used, since it appears to work on multi-dimensional data. It is an iterative function, however, and the results when it is called with multi-dimensional input are slightly different to those when it called for single values. To see the test results without these differences, here are test results using MetPy's `moist_lapse` and MetPy's `lcl` on single points. The small differences in the other outputs give us some confidence in the accuracy of the other xarray parcel functions. `wet_bulb_temperature` is not tested here because there are no existing differences in its output.

In [129]:
reload(parcel)
reload(parcel_test)

parcel.moist_lapse = parcel_test.moist_lapse_serial
parcel.lcl = parcel_test.lcl_serial
parcel.wet_bulb_temperature = metpy.calc.wet_bulb_temperature
_, _ = parcel_test.test_parcel_functions(dat=pts, virt_temp=False)

Calculating xarray results...		3.8951605320035014 s.
Calculating metpy serial results...	0.8475012890121434 s.
Differences                                                       Max abs. diff        Max rel. diff       
Mixed parcel temperature [K] (mp_temperature)                     2e-05 K              0.0%                
Mixed-parcel dewpoint [kelvin] (mp_dewpoint)                      1e-05 kelvin         0.0%                
Temperature at LCL [K] (surface_profile)                          2e-05 K              0.0%                
Level of free convection pressure [hPa] (surface_lfc_pressure)    0.00099 hPa          0.0%                
Level of free convection temperature [K] (surface_lfc_temp)       3e-05 K              0.0%                
Convective available potential energy [J kg$^{-1}$] (surface_cape) 4e-05 J kg$^{-1}$    0.0%                
Convective inhibition [J kg$^{-1}$] (surface_cin)                 0.00129 J kg$^{-1}$  -0.08%              
Environment temperature 

Small differences in moist lapse rate results introduce larger differences in CAPE and CIN calculations. Because there are many assumptions made in the calculation of CAPE and CIN (e.g. which parcel to choose, how to interpolate profiles and define intersection points), these differences are acceptable.

# Unit tests

Here we check that the xarray implementation passes all relevant unit tests from MetPy, as long as the MetPy version of `moist_lapse` is used. If so this means that the logic in all the functions (other than `moist_lapse`) is correct.

In [ ]:
reload(parcel)
reload(parcel_test)
reload(tests)

In [ ]:
parcel.load_moist_adiabat_lookups()
parcel.moist_lapse = tests.metpy_moist_lapse
tests.run_all_tests()

Now we check that the unit tests for `moist_lapse` pass using the looking table implementation, if `test_moist_lapse_uniform`'s required accuracy is changed from from 7 decimal places to 2 decimal places. The other unit tests do not all pass when the lookup table is used, because small changes in moist lapse rates have effects on profile and CAPE/CIN calculations.

In [ ]:
reload(parcel)
parcel.load_moist_adiabat_lookups()
tests.run_moist_lapse_tests_looser()

## Log interpolation of LCL environment

MetPy uses linear interpolation on pressure levels to get the environmental temperature at the LCL. I have updated my code to use logarithmic interpolation by default. Here are the differences that change makes on convective properties for the random points from the testing data used above.

In [ ]:
xr_log = parcel_test.conv_properties_xarray(dat=pts.load(), virt_temp=False, lcl_interp='log')
parcel_test.compare_results(set1=xr, set2=xr_log)

## Virtual temperature correction

MetPy does not correct for virtual temperature in buoyancy calculations as advised by [Doswell & Rasmussen 1994](https://doi.org/10.1175/1520-0434(1994)009%3C0625:TEONTV%3E2.0.CO;2). I have updated my parcel code so that by default it uses virtual temperatures in calculation of the LFC, EL, CAPE and CIN (note, not for LCL since [this should use real temperature](http://www.flame.org/~cdoswell/virtual/virtual.html).

In [ ]:
reload(parcel)
parcel.load_moist_adiabat_lookups()

Finally, we show the differences in the results of the above xarray/serial comparison when the virtual temperature correction is used. 

In [ ]:
xr_vt = parcel_test.conv_properties_xarray(dat=pts.load(), virt_temp=True, lcl_interp='linear')
parcel_test.compare_results(set1=xr, set2=xr_vt)

CAPE is increased marginally by the virtual temperature correction.

In [ ]:
plt.scatter(xr_vt.max_cape.values, xr.max_cape.values, label='Max CAPE')
plt.scatter(xr_vt.surface_cape.values, xr.surface_cape.values, label='Surface CAPE')
plt.scatter(xr_vt.mixed_cape.values, xr.mixed_cape.values, label='Mixed-parcel CAPE')
plt.axline((0,0), (1,1), color='black')
plt.title('CAPE with and without virtual temperature correction (VTC)')
plt.legend()
plt.xlabel('CAPE with VTC [J kg-1]')
plt.ylabel('CAPE without VTC [J kg-1]')
plt.show()

As expected from [Doswell & Rasmussen 1994](https://doi.org/10.1175/1520-0434(1994)009%3C0625:TEONTV%3E2.0.CO;2) the relative change in CAPE is highest for smaller values of CAPE. Relative differences range from about 8% to 12% of the corrected value.

In [ ]:
plt.scatter(xr_vt.max_cape.values, np.abs((xr.max_cape.values - xr_vt.max_cape.values) / 
                                          xr_vt.max_cape.values)*100, label='Max CAPE')
plt.scatter(xr_vt.surface_cape.values, np.abs((xr.surface_cape.values - xr_vt.surface_cape.values) / 
                                              xr_vt.surface_cape.values)*100, label='Surface CAPE')
plt.scatter(xr.mixed_cape.values, np.abs((xr.mixed_cape.values - xr_vt.mixed_cape.values) /
                                         xr_vt.mixed_cape.values)*100, label='Mixed-parcel CAPE')
plt.title('Relative differences in CAPE with and without virtual temperature correction (VTC)')
plt.legend()
plt.xlabel('CAPE with VTC [J kg-1]')
plt.ylabel('Abs. relative diff [%]')
plt.show()

The virtual temperature correction changes the LFC but not the LCL. Changes to CIN are are relatively larger than those to CAPE (relative changes are not shown here since relative changes become very large on small-magnitude CIN values).

In [ ]:
plt.scatter(xr_vt.max_cin.values, xr.max_cin.values, label='Max CIN')
plt.scatter(xr_vt.surface_cin.values, xr.surface_cin.values, label='Surface CIN')
plt.scatter(xr_vt.mixed_cin.values, xr.mixed_cin.values, label='Mixed-parcel CIN')
plt.axline((0,0), (1,1), color='black')
plt.title('CIN with and without virtual temperature correction (VTC)')
plt.legend()
plt.xlabel('CIN with VTC [J kg-1]')
plt.ylabel('CIN without VTC [J kg-1]')
plt.show()

# Benchmark for surface-based CAPE and CIN calculation

Here we benchmark the xarray implementation against the serial MetPy implementation. 

In [ ]:
reload(parcel)
reload(parcel_test)
benchmark_results = parcel_test.benchmark_cape(dat)

In [ ]:
plt.scatter(benchmark_results.pts.values, benchmark_results.serial_time.values)
plt.scatter(benchmark_results.pts.values, benchmark_results.vector_time.values)
plt.plot(benchmark_results.pts.values, benchmark_results.serial_time.values, label='MetPy serial')
plt.plot(benchmark_results.pts.values, benchmark_results.vector_time.values, label='xarray vector')
plt.xlabel('Number of points')
plt.ylabel('Processing time [s]')
plt.title('Performance comparison')
plt.legend()
plt.show()

In [ ]:
plt.scatter(benchmark_results.pts.values, benchmark_results.vector_time.values)
plt.plot(benchmark_results.pts.values, benchmark_results.vector_time.values, label='xarray vector')
plt.xlabel('Number of points')
plt.ylabel('Processing time [s]')
plt.title('xarray performance')
plt.show()

In [ ]:
plt.scatter(benchmark_results.pts.values, (benchmark_results.serial_time.values/
                                           benchmark_results.vector_time.values))
plt.plot(benchmark_results.pts.values, (benchmark_results.serial_time.values/
                                        benchmark_results.vector_time.values))
plt.xlabel('Number of points')
plt.ylabel('Speedup')
plt.title('Speedup (serial/vector)')
plt.show()